# Capstone Project - The Battle of Neighbourhoods

## Best Places in Frankfurt to open an Uzbek Restaurant

### Introduction/Business Problem

We are interested in opening an Uzbek restaurant in Frankfurt, which potentially can attract a high number of customers. Ideally, our stakeholders are interested in a **narrowed-down list of potential locations to explore further**. 

The choice of proper locations for an Uzbek restaurant in Frankfurt is very important, because Uzbek cuisine is not widely spread in Frankfurt unlike any other Asian or European cuisines, though it is very popular among European tourists visiting Uzbekistan due to its great potential in gastronomic tourism. Currently, there is only one Uzbek restaurant in Frankfurt and it is located in **Wöllstädter Str. 11, 60385 Frankfurt am Main in Bornheim neighborhood**.

Our stakeholders point out that a potentially good location for an Uzbek restaurant in Frankfurt should comply to the following criteria:

* Locations close to the city center
* Locations with low restaurant density
* Locations with few or no close Asian restaurants (competitors)
* Locations with high population density
* Locations close to tourist destinations/attractions

Our goal is to come up with a manageable list of optimal locations in Frankfurt that satisfy all of the above criteria.

### Data

According to the criteria that we have defined in our business understanding section, the following factors will influence the choice of optimal locations for an Uzbek restaurant in Frankfurt

* The closer a location to the city center, the better. The city center will be defined at **Hauptwache** (the street is called “An der Hauptwache”). **The distance between a potential location and Hauptwache** will determine the closeness criterion.
* **The number of restaurants** nearby a potential location will be counted towards the **restaurant density**. The less restaurants surround the location, the more suitable it will be.
* **The number of Asian restaurants** surrounding a given location will be counted to determine suitability of a location, as they are close competitors to the Uzbek cuisine. 
* **Neighborhoods with high population density** will be preferable for an optimal location choice, because potentially more customers can be attracted there. 
* Finally, **major tourist attractions** will be taken into account as these places also attract many foreign tourists who can potentially become customers of a nearby Uzbek restaurant.

The following data sources will be used to extract required data:

* Distances from the city center, from the main tourist attractions to potential locations will be obtained using **Google Maps API geocoding and revers geocoding**.
* The names of most popular tourist attractions in Frankfurt will be obtained from **Tripadvisor.com using web scraping** as it does not provide API access for data analysis purposes.
* The number of restaurants, their category and type as well as their geolocation information will be obtained through **Foursquare API**.
* The names of the neighbourhoods in Frankfurt and their population density will be obtained **using web scraping from Wikipedia** at https://de.wikipedia.org/wiki/Liste_der_Stadtteile_von_Frankfurt_am_Main.
